In [1]:
import pandas as pd
import numpy as np
import re
import boto3
import s3fs
from collections import defaultdict
def get_file_names(bucket_name,prefix):
    """
    Return the latest file name in an S3 bucket folder.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (folder  name).
    """
    s3_client = boto3.client('s3')
    objs = s3_client.list_objects_v2(Bucket=bucket_name)['Contents']
    shortlisted_files = []            
    for obj in objs:
        key = obj['Key']
        timestamp = obj['LastModified']
        # if key starts with folder name retrieve that key
        if key.startswith(prefix) and "parquet" in key:              
            # Adding a new key value pair
            #shortlisted_files.update( {key : timestamp} )
            shortlisted_files.append(key)
    return shortlisted_files

In [2]:
filenames=defaultdict(list)
year = ["2016","2017","2018","2019","2020"]
for yr in year:
    filenames[yr] = get_file_names(bucket_name='output-east-2-usama',prefix = 'walmart_parquet_'+yr)
    #First file is not parquet file
    filenames[yr].pop(0)
#print(filenames)

In [3]:
#List of data frames
dataframes = []
for yr in year:
    data = [pd.read_parquet("s3://output-east-2-usama/"+f) for f in filenames[yr]]
    df = pd.concat(data,ignore_index=True)
    #inserting a yr column
    df.insert(2,'year',yr)
    dataframes.append(df)
dataframes[2]

,Product,Price,year
0,None,$ to $ Go P,2018
1,None,$ to $ Go Please,2018
2,None,$ to $ Go Please enter a minimum and maximum p...,2018
3,None,$ to $ Go Please enter a minimum and maximum p...,2018
4,None,$ to $ Go Please enter a minimum and maximum p...,2018
...,...,...,...
194997,None,$ 34 . 38 Sold & shipped by AMI Ventures Inc F...,2018
194998,None,$ 66 . 67 Free shipping Pickup discount eligib...,2018
194999,None,$ 66 . 67 Free shipping Pickup discount eligib...,2018
195000,None,$ 26 . 00 Free shipping on orders over $35 Pic...,2018


In [4]:
#Do all the cleaning for each df
CleanDF=[]
for df in dataframes:
    #Dropping rows without Price information
    no_price = df[df.Price.astype(bool)]
    final = no_price[no_price.Product.astype(bool)]
    #Dropping duplicates
    dedup = final.drop_duplicates()
    Final = dedup.drop_duplicates(subset="Product")
    #Cleaning any spaces or unnecessary columns in the price field
    Final = Final.assign(Price=lambda x: x['Price'].str.replace(" ",""))
    Final = Final.assign(Price=lambda x: x['Price'].str.replace(",",""))
    #Getting price formatted
    Final = Final.assign(Price=lambda x: x['Price'].str.extract('^(\$?\d{1,20}\s*\.\s*\d{1,20})',expand=False))
    Final = Final.dropna()
    CleanDF.append(Final)

In [5]:
CleanDF[3]

,Product,Price,year
12,6FT USB Type C Cable Fast Charging Cable U,$4.89,2019
71,"Moto E4 Case, [Not Fit Moto E4 Plus] Dua",$8.99,2019
133,"Insten 13.3"" Laptop Tablet Sleeve Zip",$8.49,2019
148,Insten Universal Underwat,$7.51,2019
216,"Serial Cable, DB9 Male, U",$5.78,2019
...,...,...,...
46286,HP OMEN 17-an120nr Gaming Laptop 17,$1319.0,2019
46328,Insten Quicksand Glitter,$7.23,2019
46397,Premium Large Size Sport,$11.97,2019
47025,"Dell G3 Gaming Laptop 15.6"" Full HD, Int",$699.99,2019


In [6]:
Walmart_2016_2020 = pd.concat(CleanDF)

In [7]:
Walmart_2016_2020

,Product,Price,year
97,"HP 15-bw002wm 15.6"" Marine Blue Laptop, Window...",$199.00,2017
139,"HP 15-Bw010Nr 15.6"" Laptop, Windows 10, Window...",$299.00,2017
253,"Lenovo Legion Y520 15.6"" Gaming Laptop, Window...",$499.00,2017
279,"Refurbished Apple MacBook Pro 13.3"" LED Intel ...",$249.99,2017
352,"HP 15-bw002wm 15.6"" Marine Blue Laptop, Window...",$142.74,2017
...,...,...,...
2399,Replacement For Dell X284G Laptop Battery (56W...,$29.98,2020
2402,Replacement Dell Laptop Battery for Inspiron 1...,$37.82,2020
2405,Battery For Dell Latitude E6400 E6410 E6500 E6...,$20.69,2020
2409,eReplacements 4400mah Replacement Laptop Batte...,$47.95,2020


In [8]:
Walmart_2016_2020.to_csv("Walmart_laptops_2016_2020.csv",index=False)

In [9]:
#Plot trends for dell, HP, Lenovo, Acer, Apple? , Refurbished products
#Group them together?

In [10]:
brands =defaultdict(list)
#yr = ["2017","2018","2019","2020"]
brand_list = ["HP","Dell","Apple","Lenovo","Refurbished Laptops"]
brands['Brand'] = brand_list
brands

defaultdict(list,
            {'Brand': ['HP',
              'Dell',
              'Apple',
              'Lenovo',
              'Refurbished Laptops']})

In [11]:
prod = Walmart_2016_2020.assign(Product=lambda x: x['Product'].str.lstrip())
prod =prod.assign(Price=lambda x: x['Price'].str.replace("$",""))
prod['Price'] = pd.to_numeric(prod['Price'])

In [12]:
#Expensive laptops
high_exp = prod['Price'] > 1100
exp_level = prod[high_exp]
exp_level

,Product,Price,year
1518,"Acer Ultrabook Aspire S5-371 13.3"" Laptop, Win...",1333.92,2017
1522,"Acer Aspire Switch Alpha 12"" Laptop, touch scr...",1934.10,2017
4007,"Dell Epic Silver 17.3"" Alienware AW17R3-4175SL...",1199.00,2017
4017,"Dell Inspiron 15 Gaming Edition Black 15.6"" 75...",1949.00,2017
4028,Dell Inspiron 15 Gaming Edition 15.6 Laptop In...,1343.00,2017
...,...,...,...
1981,GIGABYTE AERO 15 Classic-,2499.00,2020
2010,"ASUS Zenbook Laptop 15.6,",1644.99,2020
2046,ASUS ROG STRIX SCAR II Gaming Lapto,1799.99,2020
2082,ASUS ROG Zephyrus Gaming Laptop 15.,1899.00,2020


In [13]:
high_level_prices=[]
years = ["2017","2018","2019","2020"]
for yr in years:
    #print(yr)
    highlevel = exp_level['year'] == yr
    highlevel_mean= exp_level.loc[highlevel, 'Price'].mean()
    high_level_prices.append(highlevel_mean)
print(high_level_prices)
#highlevel_2017 = exp_level['year'] == '2017'
#highlevel_for_2017= exp_level.loc[highlevel_2017, 'Price'].mean()
#high_level
#print("Price in 2017: ",highlevel_for_2017)

#highlevel_2018 = exp_level['year'] == '2018'
#highlevel_for_2018= exp_level.loc[highlevel_2018, 'Price'].mean()
#print("Price in 2018: ",highlevel_for_2018)

#highlevel_2019 = exp_level['year'] == '2019'
#highlevel_for_2019= exp_level.loc[highlevel_2019, 'Price'].mean()
#print("Price in 2019: ",highlevel_for_2019)

#highlevel_2020 = exp_level['year'] == '2020'
#highlevel_for_2020= exp_level.loc[highlevel_2020, 'Price'].mean()
#print("Price in 2020: ",highlevel_for_2020)

[1581.5145554789801, 1695.1099135802467, 2015.9009302325583, 1606.1583333333335]


In [15]:
#Mid-level laptops
low_mid = prod['Price'] > 800
high_mid = prod['Price'] < 1100
mid_level = prod[low_mid & high_mid]
mid_level

,Product,Price,year
740,"Teqnio ELL1103T 11.6"" Laptop, Touchscreen, 2-i...",876.00,2017
1523,"Acer G9-593-77WF 15.6"" Predator Gaming Laptop,...",912.51,2017
1528,"Acer Aspire R3-131 11.6"" Laptop, Touchscreen, ...",1019.00,2017
1534,"Acer C91054M1 15.6"" Chromebook, Chrome, Intel ...",933.70,2017
2613,"HP 17 -x020nr 17 .3"" Laptop , touch screen, Wi...",999.99,2017
...,...,...,...
43014,"ASUS ROG Strix Gaming Laptop 15.6"", Intel",829.99,2019
43244,"ASUS ROG Strix Gaming Laptop 15.6"", Int",829.99,2019
670,"Dell G3 Gaming Laptop 15.6"" Full HD, In",999.00,2020
1408,"Legion By Lenovo Y540 15.6"" Gaming Lapt",849.00,2020


In [16]:
mid_level_prices=[]
for yr in years:
    #print(yr)
    midlevel = mid_level['year'] == yr
    midlevel_mean= mid_level.loc[midlevel, 'Price'].mean()
    mid_level_prices.append(midlevel_mean)
print(mid_level_prices)
#midlevel_2017 = mid_level['year'] == '2017'
#midlevel_for_2017= mid_level.loc[midlevel_2017, 'Price'].mean()
#print("Price in 2017: ",midlevel_for_2017)

#midlevel_2018 = mid_level['year'] == '2018'
#midlevel_for_2018= mid_level.loc[midlevel_2018, 'Price'].mean()
#print("Price in 2018: ",midlevel_for_2018)

#midlevel_2019 = mid_level['year'] == '2019'
#midlevel_for_2019= mid_level.loc[midlevel_2019, 'Price'].mean()
#print("Price in 2019: ",midlevel_for_2019)

#midlevel_2020 = mid_level['year'] == '2020'
#midlevel_for_2020= mid_level.loc[midlevel_2020, 'Price'].mean()
#print("Price in 2020: ",midlevel_for_2020)

[948.3519168026103, 954.8468703703703, 932.5155172413795, 965.9966666666666]


In [17]:
#Only keep laptops more than $500 and less than 800 - "Affordable laptops"
low = prod['Price'] > 500
high = prod['Price'] < 800
affordable = prod[low & high]
affordable

,Product,Price,year
993,"HP 15-bw002wm 15.6"" Marine Blue Laptop, Window...",549.00,2017
1024,"HP 15-Bw010Nr 15.6"" Laptop, Windows 10, Window...",604.99,2017
1356,"HP SmartBuy Gray 14"" Chromebook PC with Intel ...",599.00,2017
1407,"Dell Inspiron 11 3168 11.6"" Laptop, Touchscree...",529.99,2017
1468,"Acer CB3-532-C47C 15.6"" Chromebook, Chrome OS,...",509.00,2017
...,...,...,...
448,ASUS ROG Strix Scar III G,658.80,2020
530,Dell Inspiron 15 Premium,733.95,2020
637,Dell Inspiron 14 5482 2-in-1 Touchscreen La,719.00,2020
958,"ASUS TUF Gaming Laptop, 15.6"" Full",579.99,2020


In [19]:
low_level_prices=[]
for yr in years:
    #print(yr)
    lowlevel = affordable['year'] == yr
    lowlevel_mean= affordable.loc[lowlevel, 'Price'].mean()
    low_level_prices.append(lowlevel_mean)
print(low_level_prices)

[620.5739877300614, 631.4525526838966, 663.3194444444445, 671.6216666666666]


In [20]:
#Creata a dataframe for laptops and prices over these years
#laptops = {'Type': ['Low-level', 'Mid-Level','High-Level'],'2017': [620.57,948.3,1581.5],
#           '2018':[631.45,954.8,1695.1],'2019':[663.3,932.5,2015.9],'2020':[671.6,965.99,1606.15]}
low_level_prices=[round(num, 1) for num in low_level_prices]
mid_level_prices=[round(num, 1) for num in mid_level_prices]
high_level_prices=[round(num, 1) for num in high_level_prices]
print(mid_level_prices)
print(high_level_prices)
laptops = {'Year':years,'Low-Level':low_level_prices,'Mid-Level':mid_level_prices
           ,'High-Level':high_level_prices}

laptop_prices = pd.DataFrame(laptops, columns = ['Year', 'Low-Level','Mid-Level','High-Level'])
laptop_prices
laptop_prices.to_csv("laptop_trends_2017-2020.csv",index=True)

[948.4, 954.8, 932.5, 966.0]
[1581.5, 1695.1, 2015.9, 1606.2]


In [191]:
#Walmart_2016_2020.Product.str.extract('^(Refurbished)')
#Refurbished = prod[prod.Product.str.extract('^\s*(Refurbished )')]
#Walmart_2016_2020[Walmart_2016_2020['Product'].str.contains("Apple")]
#Walmart_2016_2020['Brand']= Walmart_2016_2020.Product.str[0:200]
#Walmart_2016_2020
#Walmart_2016_2020[Walmart_2016_2020['Brand'].str.contains("Apple")]
Refurbished_Laptops = prod[prod.Product.str.startswith('Refurbished')]
Refurbished_Laptops
#Walmart_2016_2020

,Product,Price,year
279,"Refurbished Apple MacBook Pro 13.3"" LED Intel ...",249.99,2017
1384,"Refurbished HP 15-f387wm 15.6"" Laptop, Touchsc...",279.00,2017
1394,"Refurbished Samsung 11.6"" LED 16GB Chromebook...",249.99,2017
1657,"Refurbished HP 15-ay039wm 15.6"" Laptop, Window...",229.00,2017
1728,"Refurbished HP 15-f272wm Flyer Red 15.6"" Lapt...",159.00,2017
...,...,...,...
168,"Refurbished Dell Silver 14.1"" Latitude",198.00,2020
280,Refurbished Dell XPS 15 9,1484.99,2020
369,Refurbished Dell XPS 13 9,1199.95,2020
486,"Refurbished Dell 14"" E6420 Laptop PC wi",274.98,2020


In [116]:
HP_Pav = prod[prod.Product.str.startswith('HP Pavilion')]
HP_Pav = HP_Pav.assign(Price=lambda x: x['Price'].str.replace("$",""))
HP_Pav['Price'] = pd.to_numeric(HP_Pav['Price'])
#Only keep laptops more than $500
real = HP_Pav['Price'] > 500
HP_Pav = HP_Pav[real]
HP_Pav

,Product,Price,year
3107,"HP Pavilion 15-aw068nr 15.6"" Laptop, Touchscre...",994.31,2017
7140,"HP Pavilion 15-bc067nr 15.6"" Laptop, Windows 1...",1019.00,2017
8344,"HP Pavilion 15-Cc065Nr 15.6"" Laptop , Touchscr...",599.00,2017
10079,"HP Pavilion 15-au020wm 15.6"" Manhattan Gold La...",599.99,2017
15112,"HP Pavilion 15-Cc563Nr 15.6"" Laptop , Touchscr...",589.99,2017
...,...,...,...
26587,HP Pavilion 15-cr0053wm X360 Touch Lapt,699.00,2019
28047,HP Pavilion Gaming 15-CX0030NR Lapt,779.99,2019
37712,"HP Pavilion Gaming 15-CX0030NR Laptop 15.6"", I...",909.99,2019
43655,"HP Pavilion 15.6"" Power G",749.99,2019


In [120]:
HP_Pav_2017 = HP_Pav['year'] == '2017'
mean_for_2017= HP_Pav.loc[HP_Pav_2017, 'Price'].mean()
print("Price in 2017: ",mean_for_2017)
HP_Pav_2018 = HP_Pav['year'] == '2018'
mean_for_2018= HP_Pav.loc[HP_Pav_2018, 'Price'].mean()
print("Price in 2018: ",mean_for_2018)
HP_Pav_2019 = HP_Pav['year'] == '2019'
mean_for_2019= HP_Pav.loc[HP_Pav_2019, 'Price'].mean()
print("Price in 2019: ",mean_for_2019)

Price in 2017:  765.2769615384615
Price in 2018:  682.2458620689656
Price in 2019:  727.8283333333334


In [ ]:
#Create a dataframe with Brand, Average Price, Year